**SARS-CoV-2 Tracking Notebook**

In [1]:
import pandas as pd
import numpy as np
import datascience as ds
from ipywidgets import interact, interactive, Dropdown, IntSlider
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import plotly.express as px
import warnings
import plotly.io as pio
import plotly.graph_objects as go

pd.options.display.max_rows = 999
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = [18,10]
pio.templates.default = "plotly_dark"
#pio.templates.default = "none"

In [14]:
todays_date_input = input('Welcome back! What was the date yesterday? ')
todays_date = int(todays_date_input)
new_cases_input = input('How many new cases were there yesterday? ')
new_cases = int(new_cases_input)
total_deaths_input = input('What is the total death count as of yesterday? ')
total_deaths = int(total_deaths_input)
overview_df = national_month_overview()
if new_cases < overview_df['Average Cases Added per Day'].values[5]:
    print('Yesterday was a good day!')
elif new_cases > overview_df['Average Cases Added per Day'].values[5]:
    print('Yesterday was a bad day')
print('It has been ' + str(132 + todays_date) + ' since the disease appeared in the United States and ' + str((132+todays_date)-58) + ' days since shelter in place started in California.')
days_since_peak = (132 + todays_date) - 95
if new_cases < 36137:
    print('It has been ' + str((132 + todays_date) - 95) + ' days since new cases peaked on April 24th.')
else:
    print('The new case count has exceeded the previous maximum, thus resetting the clock on the curve flattening.')
this_weeks_cases = str(sum(date_frame['New Cases'].values[(132+ todays_date - 7):(132 + todays_date)]))
last_weeks_cases = str(sum(date_frame['New Cases'].values[(132 + todays_date - 15):(132 + todays_date -8)]))
week_percent_change = abs((round((((int(this_weeks_cases)-int(last_weeks_cases))/int(this_weeks_cases))*100), 1)))
weeks_percent_change_str = str(week_percent_change) + '%'
print('This week there have been ' + this_weeks_cases + ' new cases nationally.')
print('Last week, there were ' + last_weeks_cases + ' new cases nationally.')
if int(this_weeks_cases) < int(last_weeks_cases):
    print('This week has been better than the last by ' + str(int(last_weeks_cases) - int(this_weeks_cases)) + ' cases, or a decrease in new cases of ' + weeks_percent_change_str + '.')
elif int(this_weeks_cases) > int(last_weeks_cases):
    print('This week has been worse than last by ' + str(int(this_weeks_cases) - int(last_weeks_cases)) + ' cases or an increase in new cases of ' + weeks_percent_change_str + '.')
print('Please update the new cases for California and redownload the CSVs.')

Welcome back! What was the date yesterday? 23
How many new cases were there yesterday? 33315
What is the total death count as of yesterday? 119719
Yesterday was a bad day
It has been 155 since the disease appeared in the United States and 97 days since shelter in place started in California.
It has been 60 days since new cases peaked on April 24th.
This week there have been 201564 new cases nationally.
Last week, there were 145083 new cases nationally.
This week has been worse than last by 56481 cases or an increase in new cases of 28.0%.
Please update the new cases for California and redownload the CSVs.


In [ ]:
#mem_yest_new_cases = 33315
#mem_yest_deaths = 119719

In [3]:
def date_maker(month):
    if month == 1:
        mes = 'January'
        mes_len = 31
    if month == 2:
        mes = 'February'
        mes_len = 29
    elif month == 3:
        mes = 'March'
        mes_len = 31
    elif month == 4:
        mes = 'April'
        mes_len = 30
    elif month == 5:
        mes = 'May'
        mes_len = 31
    elif month == 6:
        mes = 'June'
        mes_len = todays_date
    if month == 1:
        start_date = 21
    else: 
        start_date = 1
    year = 2020
    empty_range = []
    for i in range(start_date, mes_len + 1):
        empty_range.append(mes + f" {i:d}" + ', ' + str(year))
    return empty_range

In [4]:
jan = date_maker(1)
feb = date_maker(2)
mar = date_maker(3)
apr = date_maker(4)
may = date_maker(5)
jun = date_maker(6)
all_dates = jan + feb + mar + apr + may + jun

In [5]:
jan_new_cases = [1, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1]
feb_new_cases = [1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 4, 5]
mar_new_cases = [18, 15, 28, 26, 64, 77, 101, 144, 148, 291, 269, 393, 565, 662, 676, 872, 1291, 2410, 3948, 5417, 6271, 8631, 10410, 9939, 12226, 17050, 19046, 20093, 19118, 20463, 25305]
apr_new_cases = [26756, 28879, 32525, 33725, 25717, 29359, 30512, 32282, 34243, 33578, 31869, 28057, 24685, 25987, 29465, 32076, 30915, 28084, 26013, 25132, 25178, 29246, 31277, 35930, 34782, 26858, 22042, 24359, 25869, 29637]
may_new_cases = [32964, 30531, 25382, 21278, 22226, 24805, 27514, 27095, 27421, 20697, 17779, 21486, 20549, 26762, 25382, 24527, 19731, 21109, 19662, 22368, 25028, 22206, 22717, 20253, 18554, 16682, 18636, 23016, 23575, 21385, 21686]
jun_new_cases = [16040, 20039, 22162, 21004, 23392, 20865, 18535, 16301, 17180, 20244, 17849, 24421, 25891, 21191, 18307, 23047, 23624, 27082, 30927, 31963, 28095, 26558, new_cases]
new_cases_over_time = jan_new_cases + feb_new_cases + mar_new_cases + apr_new_cases + may_new_cases + jun_new_cases

In [6]:
def total_cases_national(lst):
    case_list = []
    for i in lst:
        if len(case_list)>0:
            case_list.append(case_list[-1]+i)
        else:
            case_list.append(i)
    return case_list

total_cases_over_time = total_cases_national(new_cases_over_time)
pre_cases_per_million = list(total_cases_over_time)
cases_per_million = [int((element / 328200000) * 1000000) for element in pre_cases_per_million]

In [7]:
jan_deaths = list([0] * 11)
feb_deaths = list([0] * 29)
mar_deaths = [1, 2, 6, 9, 11, 12, 14, 17, 21, 26, 28, 30, 40, 47, 57, 69, 85, 108, 150, 150, 260,340, 471, 590,801, 1050, 1296, 1707, 2191, 2509, 3170]
apr_deaths = [4079, 5138, 6053, 7157, 8501, 9647, 10989, 12895, 14817, 16690, 18777, 20608, 22108, 23649, 26057, 30985, 33284, 37054, 38910, 40682, 42539, 45063, 46784, 49963, 51017, 53189, 54876, 56245, 58355, 60966]
may_deaths = [63006, 65068, 66385, 67682, 68934, 71078, 73431, 75670, 77180, 78794, 79528, 80684, 82387, 84133, 85906, 87568, 88754, 89562, 90353, 91121, 93439, 94702, 96007, 97087, 97720, 98220, 98916, 100442, 101617, 102836, 103781]
jun_deaths = [104383, 105147, 106181, 107175, 108211, 109143, 109802, 110514, 111007, 112006, 112924, 113820, 114669, 115436, 115732, 116127, 116963, 117717, 118434, 119112, 119719, 119975, total_deaths]
total_deaths_over_time = jan_deaths + feb_deaths + mar_deaths + apr_deaths + may_deaths + jun_deaths

def death_rate_strings():
    death_rate_range = []
    for i in range(0, 131 + todays_date, 1):
        death_rate_range.append(round((((total_deaths_over_time[i])/total_cases_over_time[i]) * 100), 2))
        death_rate_list = list(map('{}%'.format, death_rate_range))
    return death_rate_list

all_death_rates = death_rate_strings()
todays_death_rate = str(round(int(total_deaths)/int(total_cases_over_time[131 + todays_date])*100, 2))
todays_death_rate_str = todays_death_rate + '%'
all_death_rates.insert(132+todays_date, todays_death_rate_str)

new_deaths = list(np.diff(total_deaths_over_time))
new_deaths.insert(0,0)

In [8]:
def rate_of_change_data():
    percent_range = []
    for i in range(0, 131 + todays_date, 1):
        percent_range.append(round((((new_cases_over_time[i + 1] + 1) - (new_cases_over_time[i] + 1))/(new_cases_over_time[i] +1)*100), 2))
        percent_list = list(map("{}%".format, percent_range))
    return percent_list

percent_data = rate_of_change_data()
percent_data.insert(0,'0.0%')

In [9]:
date_frame = pd.DataFrame({
    'Date': all_dates,
    'New Cases': new_cases_over_time,
    'New Cases Percent Change' : percent_data,
    'Total Cases': total_cases_over_time,
    'Cases per Million': cases_per_million,
    'Total Deaths': total_deaths_over_time,
    'Death Rate': all_death_rates,
    "New Deaths": new_deaths
})
date_frame

,Date,New Cases,New Cases Percent Change,Total Cases,Cases per Million,Total Deaths,Death Rate,New Deaths
0,"January 21, 2020",1,0.0%,1,0,0,0.0%,0
1,"January 22, 2020",0,-50.0%,1,0,0,0.0%,0
2,"January 23, 2020",0,0.0%,1,0,0,0.0%,0
3,"January 24, 2020",1,100.0%,2,0,0,0.0%,0
4,"January 25, 2020",1,0.0%,3,0,0,0.0%,0
5,"January 26, 2020",2,50.0%,5,0,0,0.0%,0
6,"January 27, 2020",0,-66.67%,5,0,0,0.0%,0
7,"January 28, 2020",0,0.0%,5,0,0,0.0%,0
8,"January 29, 2020",0,0.0%,5,0,0,0.0%,0
9,"January 30, 2020",1,100.0%,6,0,0,0.0%,0


In [10]:
graph_percents = []
for i in range(0, 131 + todays_date, 1):
    graph_percents.append(round((((new_cases_over_time[i + 1] + 1) - (new_cases_over_time[i] + 1))/(new_cases_over_time[i] +1)*100), 2))
graph_percents.insert(0,0)



days_since = np.arange(1, 133 + todays_date)

graph_death_rates = []
for i in range(0, 131 + todays_date, 1):
    graph_death_rates.append(round((((total_deaths_over_time[i])/total_cases_over_time[i]) * 100), 2))
graph_death_rates.insert(131 + todays_date, round(total_deaths/int(total_cases_over_time[131 + todays_date])*100, 2))
        
data_df = pd.DataFrame({
    'Days Since the Pandemic Started': days_since,
    'New Cases over Time': new_cases_over_time,
    'Percent Change in Cases': graph_percents,
    'Total Cases over Time': total_cases_over_time,
    'Cases per Million': cases_per_million,
    'Total Deaths': total_deaths_over_time,
    'Death Rate': graph_death_rates,
    'New Deaths': new_deaths
})

data_df = data_df.astype(float)

def plotly_graph_national(choice):
    fig = px.line(data_df, x="Days Since the Pandemic Started", y=str(choice), title='SARS-CoV-2 2020 Pandemic in the United States')
    return fig.show()

In [11]:
Graphs = list(data_df)
Graphs.remove('Days Since the Pandemic Started')
interact(lambda Graphs:
         plotly_graph_national(choice = Graphs),
        Graphs = Dropdown(options = Graphs))
print('')

interactive(children=(Dropdown(description='Graphs', options=('New Cases over Time', 'Percent Change in Cases'…

In [12]:
def national_month_snapshot():
    dates = ("January 31st", "February 29th", "March 31st", 'April 30th', 'May 31st', 'June ' + str(todays_date)) 
    confirmed_cases = (total_cases_over_time[10], total_cases_over_time[39], total_cases_over_time[70], total_cases_over_time[100], total_cases_over_time[131], int(total_cases_over_time[131 + todays_date]))    
    new_cases_overview = (jan_new_cases[10], feb_new_cases[28], mar_new_cases[30], apr_new_cases[29], may_new_cases[30], new_cases)
    deaths = (max(jan_deaths), max(feb_deaths), max(mar_deaths), max(apr_deaths), max(may_deaths), total_deaths)
    pre_cases_per_million = list(confirmed_cases)
    death_rate = (all_death_rates[10], all_death_rates[39], all_death_rates[69], all_death_rates[99], all_death_rates[131], all_death_rates[131 + todays_date])
    cases_per_million = [int((element / 328200000) * 1000000) for element in pre_cases_per_million]
    new_deaths_snapshot = (new_deaths[10], new_deaths[38], new_deaths[70], new_deaths[99], new_deaths[131], new_deaths[131+todays_date])
    df = pd.DataFrame({
        "Month End Snapshot": dates,
        "Confirmed Cases": confirmed_cases,
        "Cases per Million": cases_per_million,
        'New Cases': new_cases_overview,
        "Deaths": deaths,
        'Death Rate': death_rate,
        'New Deaths': new_deaths_snapshot
    })
    return df
national_month_snapshot()

,Month End Snapshot,Confirmed Cases,Cases per Million,New Cases,Deaths,Death Rate,New Deaths
0,January 31st,7,0,1,0,0.0%,0
1,February 29th,24,0,5,0,0.0%,0
2,March 31st,185991,566,25305,3170,1.56%,661
3,April 30th,1061028,3232,29637,60966,5.66%,2110
4,May 31st,1774034,5405,21686,103781,5.85%,945
5,June 23,2302066,7014,33315,119719,5.2%,-256


In [13]:
def national_month_overview():
    months = ('January', 'February', 'March', 'April', 'May', 'June') 
    confirmed_cases = (total_cases_over_time[10], total_cases_over_time[39], total_cases_over_time[70], total_cases_over_time[100], total_cases_over_time[131], int(total_cases_over_time[131 + todays_date]))    
    cases_added = ds.make_array(7, 17, (185991-24), (1061028-185991), (1776463 - 1061028), ((total_cases_over_time[131 + todays_date])-1776463))
    month_length = ds.make_array(len(jan), len(feb), len(mar), len(apr), len(may), len(jun))
    average_cases_added_per_day = np.round((cases_added)/(month_length), 2)
    pre_cases_per_million = list(confirmed_cases)
    overview_cases_per_million = [int((element / 328200000) * 1000000) for element in pre_cases_per_million]
    change_in_cases_per_million = (0, 0, 566, (3237 - 566), 5418-(3237), cases_per_million[131+todays_date] - 5418)
    deaths = (max(jan_deaths), max(feb_deaths), max(mar_deaths), max(apr_deaths), max(may_deaths), (total_deaths))
    deaths_added = (0,0, 3170, (60966-3170), (103781 - 60966), (total_deaths) - 103781)
    average_deaths_added = np.round((deaths_added/month_length),2)
    death_rate = (all_death_rates[10], all_death_rates[39], all_death_rates[69], all_death_rates[99], all_death_rates[131], all_death_rates[131 + todays_date])
    int_death_rates = (graph_death_rates[10], graph_death_rates[39], graph_death_rates[69], graph_death_rates[99], graph_death_rates[131], graph_death_rates[131 + todays_date])
    death_rates_diff = np.round(np.diff(int_death_rates),2)
    inserted_death_rates = np.insert(death_rates_diff,0,0)
    change_in_death_rate = list(map("{}%".format, inserted_death_rates))
    overview_df = pd.DataFrame({
        'Month' : months,
        'Confirmed Cases': confirmed_cases,
        'Cases Added': cases_added,
        'Average Cases Added per Day': average_cases_added_per_day,
        'Cases per Million': overview_cases_per_million,
        'Change in Cases per Million': change_in_cases_per_million,
        'Deaths': deaths,
        'Deaths Added': deaths_added,
        'Average Deaths Added per Day': average_deaths_added,
        'Death Rate': death_rate,
        'Change in Death Rate': change_in_death_rate
    })
    return overview_df

national_month_overview()

,Month,Confirmed Cases,Cases Added,Average Cases Added per Day,Cases per Million,Change in Cases per Million,Deaths,Deaths Added,Average Deaths Added per Day,Death Rate,Change in Death Rate
0,January,7,7,0.64,0,0,0,0,0.00,0.0%,0.0%
1,February,24,17,0.59,0,0,0,0,0.00,0.0%,0.0%
2,March,185991,185967,5998.94,566,566,3170,3170,102.26,1.56%,1.56%
3,April,1061028,875037,29167.90,3232,2671,60966,57796,1926.53,5.66%,4.1%
4,May,1774034,715435,23078.55,5405,2181,103781,42815,1381.13,5.85%,0.19%
5,June,2302066,525603,22852.30,7014,1596,119719,15938,692.96,5.2%,-0.65%


In [15]:
ca_jan_tot_cases = [1, 1, 2, 2, 2, 2, 3]
ca_feb_tot_cases = [3, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 9, 9, 9, 11, 11, 26, 26, 27, 28]
ca_mar_tot_cases = [33, 38, 45, 55, 67, 81, 100, 112, 172, 179, 202, 252, 320, 381, 478, 588, 732, 893, 1067, 1283, 1544, 1851, 2240, 2644, 3183, 4060, 4914, 5565, 6266, 7421, 8582]
ca_apr_tot_cases = [9816, 11190, 12569, 13796, 15076, 16284, 17540, 19043, 20191, 21366, 22421, 23323, 24334, 25758, 27107, 28142, 29398, 30829, 31544, 33862, 35844, 37573, 39534, 41368, 42590, 43691, 45208, 46570, 48904, 50470]
ca_may_tot_cases = [52318, 53753, 55072, 56333, 58848, 60787, 62481, 64616, 66824, 68051, 69514, 71150, 73218, 74947, 77015, 78933, 80366, 81493, 83981, 86125, 88488, 90801, 92815, 94743, 97017, 99925, 101874, 104071, 107043, 110100, 113114]
ca_jun_tot_cases = [115643, 118081, 120407, 122917, 126510, 129147, 131997, 134287, 137245, 140139, 143709, 147285, 150434, 152953, 155662, 159131, 163381, 167135, 170843, 174854, 178807, 184620, 191039]
ca_total_cases = ca_jan_tot_cases + ca_feb_tot_cases + ca_mar_tot_cases + ca_apr_tot_cases + ca_may_tot_cases + ca_jun_tot_cases
ca_new_cases = np.diff(ca_total_cases)
ca_new_case_convert = list(ca_new_cases)
ca_new_case_convert.insert(0,1)

def ca_rate_of_change_data():
    percent_range = []
    for i in range(0, 126 + todays_date, 1):
        percent_range.append(round((((ca_new_cases[i + 1] + 1) - (ca_new_cases[i] + 1))/(ca_new_cases[i] +1)*100), 2))
        percent_list = list(map("{}%".format, percent_range))
    return percent_list

ca_percent_data = ca_rate_of_change_data()
ca_percent_data.insert(0,'0.0%')
ca_percent_data.insert(0,'0.0%')


ca_pre_cases_per_million = list(ca_total_cases)
ca_cases_per_million = [int((element / 39510000) * 1000000) for element in ca_pre_cases_per_million]

In [16]:
#https://docs.google.com/spreadsheets/d/19jq3L3hLWNObdouq1Mdq21S8tz74GbEY7CDG4CwI5Ng/edit?usp=sharing
HH = {'fa':'19jq3L3hLWNObdouq1Mdq21S8tz74GbEY7CDG4CwI5Ng',
        }
Sheets = HH

def dataframe_from_gspreadsheet(sheet_name, key):
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))
    df = pd.read_csv(url)
    return df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)


states = dataframe_from_gspreadsheet("Covid19 Data", HH['fa'])
ca_deaths = states[states['state'] == 'California']
ca_death_tuple = tuple(ca_deaths['deaths'])

def ca_death_rate_strings():
    ca_death_rate_range = []
    for i in range(0, len(ca_death_tuple), 1):
        ca_death_rate_range.append(round(((ca_death_tuple[i]/ca_total_cases[i])* 100), 2))
        ca_death_rate_list = list(map('{}%'.format, ca_death_rate_range))
    return ca_death_rate_list
ca_death_rates = ca_death_rate_strings()

ca_new_deaths = list(np.diff(ca_death_tuple))
ca_new_deaths.insert(0,0)

In [17]:
def ca_jan_label(month):
    if month == 1:
        mes = 'January'
        mes_len = 31
    if month == 1:
        start_date = 25
    else: 
        start_date = 1
    year = 2020
    empty_range = []
    for i in range(start_date, mes_len + 1, 1):
        empty_range.append(mes + f" {i:d}" + ', ' + str(year))
    return empty_range

ca_jan = ca_jan_label(1)
ca_dates = ca_jan + feb + mar + apr + may + jun

pd.options.display.max_rows = 999
ca_date_frame = pd.DataFrame({
    'Date': ca_dates,
    'New Cases': ca_new_case_convert,
    'New Cases Percent Change': ca_percent_data,
    'Total Cases': ca_total_cases,
    'Cases per Million': ca_cases_per_million,
    'Total Deaths': ca_death_tuple,
    'Death Rate': ca_death_rates,
    'New Deaths': ca_new_deaths
})
ca_date_frame

,Date,New Cases,New Cases Percent Change,Total Cases,Cases per Million,Total Deaths,Death Rate,New Deaths
0,"January 25, 2020",1,0.0%,1,0,0,0.0%,0
1,"January 26, 2020",0,0.0%,1,0,0,0.0%,0
2,"January 27, 2020",1,100.0%,2,0,0,0.0%,0
3,"January 28, 2020",0,-50.0%,2,0,0,0.0%,0
4,"January 29, 2020",0,0.0%,2,0,0,0.0%,0
5,"January 30, 2020",0,0.0%,2,0,0,0.0%,0
6,"January 31, 2020",1,100.0%,3,0,0,0.0%,0
7,"February 1, 2020",0,-50.0%,3,0,0,0.0%,0
8,"February 2, 2020",3,300.0%,6,0,0,0.0%,0
9,"February 3, 2020",0,-75.0%,6,0,0,0.0%,0


In [18]:
ca_graph_percents = []
for i in range(0, 126 + todays_date, 1):
    ca_graph_percents.append(round((((ca_new_cases[i + 1] + 1) - (ca_new_cases[i] + 1))/(ca_new_cases[i] +1)*100), 2))

ca_death_rate_graph = []
for i in range(0, len(ca_death_tuple), 1):
    ca_death_rate_graph.append(round(((ca_death_tuple[i]/ca_total_cases[i])* 100), 2))
ca_graph_death_rate_pre = ca_death_rate_graph


    
ca_graph_percents.insert(0,0)
ca_graph_percents.insert(0,0)
ca_days_since = np.arange(1, 129 + todays_date)
ca_df = pd.DataFrame({
    'Days Since the Pandemic Started': ca_days_since,
    'New Cases over Time': ca_new_case_convert,
    'Percent Change in Cases': ca_graph_percents,
    'Total Cases over Time': ca_total_cases,
    'Total Deaths': ca_death_tuple,
    'Death Rate': ca_graph_death_rate_pre,
    'New Deaths': ca_new_deaths
})
ca_df=ca_df.astype(float)

def plotly_ca_rate_of_change_graph(choice):
    fig = px.line(ca_df, x = 'Days Since the Pandemic Started', y = str(choice), title = 'SARS-CoV-2 2020 Pandemic in California')
    return fig.show()

In [19]:
Graphs = list(ca_df)
Graphs.remove('Days Since the Pandemic Started')
interact(lambda Graphs:
         plotly_ca_rate_of_change_graph(choice = Graphs),
        Graphs = Dropdown(options = Graphs))
print('')

interactive(children=(Dropdown(description='Graphs', options=('New Cases over Time', 'Percent Change in Cases'…

In [20]:
def california_month_overview():
    months = ('January', 'February', 'March', 'April', 'May', 'June')
    confirmed_cases = (max(ca_jan_tot_cases), max(ca_feb_tot_cases), max(ca_mar_tot_cases), max(ca_apr_tot_cases), max(ca_may_tot_cases), max(ca_jun_tot_cases))
    cases_added = ds.make_array(3, 25, (8582-28), (50470-8582), (max(ca_may_tot_cases) - 50470), (max(ca_jun_tot_cases) - 113114))
    month_length = ds.make_array(len(ca_jan), len(feb), len(mar), len(apr), len(may), len(jun))
    average_cases_added_per_day = np.round((cases_added)/(month_length), 2)
    pre_cases_per_million = list(confirmed_cases)
    overview_cases_per_million = [int((element/39510000) * 1000000) for element in pre_cases_per_million]
    change_in_cases_per_million = (0, 0, 217, (1277 - 217), 2862-(1277), ca_cases_per_million[127+todays_date] - 2862)
    overview_deaths = (ca_death_tuple[7], ca_death_tuple[36], ca_death_tuple[67], ca_death_tuple[97], ca_death_tuple[127], ca_death_tuple[127 + todays_date])
    overview_deaths_added = (0, 0, 212, 1935, 4242 - 2147, ca_death_tuple[127+todays_date] - 4242)
    ca_average_deaths_added = np.round((overview_deaths_added/month_length),2)
    ca_death_rate_overview = (ca_death_rates[5], ca_death_rates[34], ca_death_rates[66], ca_death_rates[96], ca_death_rates[127], ca_death_rates[127+todays_date])
    ca_death_rate_tuples = ((ca_graph_death_rate_pre[5], ca_graph_death_rate_pre[34], ca_graph_death_rate_pre[66], ca_graph_death_rate_pre[96], ca_graph_death_rate_pre[127], ca_graph_death_rate_pre[127 + todays_date]))
    change_in_ca_death_rate_overview = np.round(np.diff(ca_death_rate_tuples),2)
    final_change_in_ca_death_rate = np.insert(change_in_ca_death_rate_overview, 0,0)
    change_in_ca_death_rate = list(map("{}%".format, final_change_in_ca_death_rate))
    overview_df = pd.DataFrame({
        'Month' : months,
        'Confirmed Cases': confirmed_cases,
        'Cases Added': cases_added,
        'Average Cases Added per Day': average_cases_added_per_day,
        'Cases per Million': overview_cases_per_million,
        'Change in Cases per Million': change_in_cases_per_million,
        'Deaths': overview_deaths,
        'Deaths Added': overview_deaths_added,
        'Average Deaths Added per Day': ca_average_deaths_added,
        'Death Rate': ca_death_rate_overview,
        'Change in Death Rate': change_in_ca_death_rate
    })
    return overview_df
california_month_overview()

,Month,Confirmed Cases,Cases Added,Average Cases Added per Day,Cases per Million,Change in Cases per Million,Deaths,Deaths Added,Average Deaths Added per Day,Death Rate,Change in Death Rate
0,January,3,3,0.43,0,0,0,0,0.00,0.0%,0.0%
1,February,28,25,0.86,0,0,0,0,0.00,0.0%,0.0%
2,March,8582,8554,275.94,217,217,212,212,6.84,2.13%,2.13%
3,April,50470,41888,1396.27,1277,1060,2147,1935,64.50,4.08%,1.95%
4,May,113114,62644,2020.77,2862,1585,4242,2095,67.58,3.75%,-0.33%
5,June,191039,77925,3388.04,4835,1973,5637,1395,60.65,2.95%,-0.8%
